In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load and preprocess the data
data = pd.read_excel('IND.xlsx')
time_series = data[['year', 'debt']].set_index('year')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 5
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Step 3: Train the model
model.fit(X_train, y_train, batch_size=1, epochs=20)

# Step 4: Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Step 5: Evaluate the model
# Calculate MAE, MSE, RMSE, and MAPE
mae = mean_absolute_error(y_test_actual, predictions)
mse = mean_squared_error(y_test_actual, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Epoch 1/20


c:\Users\dhevi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0237
Epoch 2/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0176
Epoch 3/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0043   
Epoch 4/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0011    
Epoch 5/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0012   
Epoch 6/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0013
Epoch 7/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0014   
Epoch 8/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014    
Epoch 9/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014
Epoch 10/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0012    
Epoch 11/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0016
Epoch 12/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0019   
Epoch 13/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0011   
Epoch 14/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0012
Epoch 15/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0016
Epo

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load and preprocess the data
data = pd.read_excel('IND.xlsx')
time_series = data[['year', 'debt']].set_index('year')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Increase sequence length to capture more historical context
seq_length = 10  # Increased from 5 to 10
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Build an improved LSTM model
model = Sequential()

# First LSTM layer with more units and return sequences
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Second LSTM layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Dense layer for output
model.add(Dense(1))

# Compile the model with a lower learning rate
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)  # Lower learning rate
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Step 3: Train the model with more epochs
history = model.fit(X_train, y_train, batch_size=16, epochs=50, validation_data=(X_test, y_test))

# Step 4: Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Step 5: Evaluate the model
# Calculate MAE, MSE, RMSE, and MAPE
mae = mean_absolute_error(y_test_actual, predictions)
mse = mean_squared_error(y_test_actual, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Epoch 1/50


c:\Users\dhevi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 300ms/step - loss: 0.0707 - val_loss: 0.3372
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0302 - val_loss: 0.0492
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0124 - val_loss: 0.0132
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0196 - val_loss: 0.0097
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0105 - val_loss: 0.0068
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0082 - val_loss: 0.0075
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0102 - val_loss: 0.0304
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0044 - val_loss: 0.2264
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0050 - val_loss: 0.5916
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0081 - val_loss: 0.4438
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0039 - val_loss: 0.2524
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0055 - val_loss: 0.1794
Epoch 13/50

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load and preprocess the data
data = pd.read_excel('IND.xlsx')
time_series = data[['year', 'debt']].set_index('year')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Increase sequence length to capture more historical context
seq_length = 10  # Experiment with different values
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Build an improved LSTM model
model = Sequential()

# First LSTM layer with more units and return sequences
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Second LSTM layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Dense layer for output
model.add(Dense(1))

# Compile the model with a lower learning rate
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)  # Lower learning rate
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Step 3: Train the model with more epochs
history = model.fit(X_train, y_train, batch_size=16, epochs=100, validation_data=(X_test, y_test))

# Step 4: Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Step 5: Evaluate the model
# Calculate MAE, MSE, RMSE, and MAPE
mae = mean_absolute_error(y_test_actual, predictions)
mse = mean_squared_error(y_test_actual, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual) )* 100

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Epoch 1/100


c:\Users\dhevi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 319ms/step - loss: 0.0642 - val_loss: 0.3699
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0283 - val_loss: 0.0954
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0142 - val_loss: 0.0038
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0240 - val_loss: 0.0118
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0142 - val_loss: 0.0052
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0077 - val_loss: 0.0196
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0126 - val_loss: 0.0152
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0101 - val_loss: 0.0094
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0056 - val_loss: 0.1123
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0044 - val_loss: 0.2988
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0063 - val_loss: 0.2508
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0032 - val_loss: 0.1334


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load and preprocess the data
data = pd.read_excel('IND.xlsx')
time_series = data[['year', 'debt']].set_index('year')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Increase sequence length to capture more historical context
seq_length = 10  # Experiment with different values
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Build an improved LSTM model
model = Sequential()

# First LSTM layer with more units and return sequences
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Second LSTM layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Dense layer for output
model.add(Dense(1))

# Compile the model with a lower learning rate
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)  # Lower learning rate
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Step 3: Train the model with more epochs
history = model.fit(X_train, y_train, batch_size=16, epochs=200, validation_data=(X_test, y_test))

# Step 4: Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Step 5: Evaluate the model
# Calculate MAE, MSE, RMSE, and MAPE
mae = mean_absolute_error(y_test_actual, predictions)
mse = mean_squared_error(y_test_actual, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual) )* 100

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Epoch 1/200


c:\Users\dhevi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 311ms/step - loss: 0.0633 - val_loss: 0.3165
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0331 - val_loss: 0.0496
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0179 - val_loss: 0.0033
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0148 - val_loss: 0.0055
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0083 - val_loss: 0.0073
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0089 - val_loss: 0.0099
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0072 - val_loss: 0.0923
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0033 - val_loss: 0.3290
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0041 - val_loss: 0.3721
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0034 - val_loss: 0.3502
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0036 - val_loss: 0.5050
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0035 - val_loss: 0.5555


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load and preprocess the data
data = pd.read_excel('IND.xlsx')
time_series = data[['year', 'debt']].set_index('year')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Increase sequence length to capture more historical context
seq_length = 10  # Experiment with different values
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Build an improved LSTM model
model = Sequential()

# First LSTM layer with more units and return sequences
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Second LSTM layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Dense layer for output
model.add(Dense(1))

# Compile the model with a lower learning rate
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)  # Lower learning rate
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Step 3: Train the model with more epochs
history = model.fit(X_train, y_train, batch_size=16, epochs=500, validation_data=(X_test, y_test))

# Step 4: Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Step 5: Evaluate the model
# Calculate MAE, MSE, RMSE, and MAPE
mae = mean_absolute_error(y_test_actual, predictions)
mse = mean_squared_error(y_test_actual, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual) )* 100

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Epoch 1/500


c:\Users\dhevi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 343ms/step - loss: 0.0636 - val_loss: 0.3518
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0330 - val_loss: 0.0591
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0167 - val_loss: 0.0060
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0169 - val_loss: 0.0105
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0108 - val_loss: 0.0056
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0087 - val_loss: 0.0071
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0109 - val_loss: 0.0229
Epoch 8/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0060 - val_loss: 0.1764
Epoch 9/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0045 - val_loss: 0.3608
Epoch 10/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0045 - val_loss: 0.2164
Epoch 11/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0030 - val_loss: 0.1459
Epoch 12/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0040 - val_loss: 0.2068


In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load and preprocess the data
data = pd.read_excel('IND.xlsx')
time_series = data[['year', 'debt']].set_index('year')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Increase sequence length to capture more historical context
seq_length = 10  # Experiment with different values
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Build an improved LSTM model
model = Sequential()

# First LSTM layer with more units and return sequences
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Second LSTM layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Dense layer for output
model.add(Dense(1))

# Compile the model with a lower learning rate
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)  # Lower learning rate
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Step 3: Train the model with more epochs
history = model.fit(X_train, y_train, batch_size=16, epochs=1000, validation_data=(X_test, y_test))

# Step 4: Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Step 5: Evaluate the model
# Calculate MAE, MSE, RMSE, and MAPE
mae = mean_absolute_error(y_test_actual, predictions)
mse = mean_squared_error(y_test_actual, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual) )* 100

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Epoch 1/1000


c:\Users\dhevi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 320ms/step - loss: 0.0481 - val_loss: 0.2137
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0155 - val_loss: 0.0065
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0178 - val_loss: 0.0043
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0130 - val_loss: 0.0103
Epoch 5/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0106 - val_loss: 0.0189
Epoch 6/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0101 - val_loss: 0.0092
Epoch 7/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0084 - val_loss: 0.0364
Epoch 8/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0054 - val_loss: 0.2496
Epoch 9/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0079 - val_loss: 0.3127
Epoch 10/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0041 - val_loss: 0.1842
Epoch 11/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0029 - val_loss: 0.1388
Epoch 12/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0045 - val_lo

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load and preprocess the data
data = pd.read_excel('IND.xlsx')
time_series = data[['year', 'debt']].set_index('year')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Increase sequence length to capture more historical context
seq_length = 10  # Experiment with different values
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Build an improved LSTM model
model = Sequential()

# First LSTM layer with more units and return sequences
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Second LSTM layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Dense layer for output
model.add(Dense(1))

# Compile the model with RMSprop optimizer
from tensorflow.keras.optimizers import RMSprop
optimizer = RMSprop(learning_rate=0.001)  # RMSprop with a lower learning rate
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Step 3: Train the model with more epochs
history = model.fit(X_train, y_train, batch_size=16, epochs=500, validation_data=(X_test, y_test))

# Step 4: Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Step 5: Evaluate the model
# Calculate MAE, MSE, RMSE, and MAPE
mae = mean_absolute_error(y_test_actual, predictions)
mse = mean_squared_error(y_test_actual, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Epoch 1/500


c:\Users\dhevi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 321ms/step - loss: 0.0669 - val_loss: 0.3566
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0227 - val_loss: 0.0949
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0220 - val_loss: 0.1795
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0153 - val_loss: 0.0208
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0154 - val_loss: 0.0501
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0135 - val_loss: 0.0220
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0092 - val_loss: 0.0160
Epoch 8/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0075 - val_loss: 0.1084
Epoch 9/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0108 - val_loss: 0.0075
Epoch 10/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0061 - val_loss: 0.0448
Epoch 11/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0048 - val_loss: 0.0156
Epoch 12/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0053 - val_loss: 0.2019


In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load and preprocess the data
data = pd.read_excel('IND.xlsx')
time_series = data[['year', 'debt']].set_index('year')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Increase sequence length to capture more historical context
seq_length = 10  # Experiment with different values
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Build an improved LSTM model
model = Sequential()

# First LSTM layer with more units and return sequences
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Second LSTM layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Dense layer for output
model.add(Dense(1))

# Compile the model with RMSprop optimizer
from tensorflow.keras.optimizers import RMSprop
optimizer = RMSprop(learning_rate=0.001)  # RMSprop with a lower learning rate
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Step 3: Train the model with more epochs
history = model.fit(X_train, y_train, batch_size=16, epochs=700, validation_data=(X_test, y_test))

# Step 4: Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Step 5: Evaluate the model
# Calculate MAE, MSE, RMSE, and MAPE
mae = mean_absolute_error(y_test_actual, predictions)
mse = mean_squared_error(y_test_actual, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Epoch 1/700


c:\Users\dhevi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 319ms/step - loss: 0.0541 - val_loss: 0.1968
Epoch 2/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0181 - val_loss: 0.1291
Epoch 3/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0158 - val_loss: 0.0284
Epoch 4/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0097 - val_loss: 0.0261
Epoch 5/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0173 - val_loss: 0.0035
Epoch 6/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0077 - val_loss: 0.0077
Epoch 7/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0056 - val_loss: 0.0196
Epoch 8/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0058 - val_loss: 0.0264
Epoch 9/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0047 - val_loss: 0.1806
Epoch 10/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0088 - val_loss: 0.1145
Epoch 11/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0052 - val_loss: 0.1116
Epoch 12/700
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0057 - val_loss: 0.0420


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load and preprocess the data
data = pd.read_excel('IND.xlsx')
time_series = data[['year', 'debt']].set_index('year')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Increase sequence length to capture more historical context
seq_length = 10  # Experiment with different values
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Build an improved LSTM model
model = Sequential()

# First LSTM layer with more units and return sequences
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Second LSTM layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Dense layer for output
model.add(Dense(1))

# Compile the model with RMSprop optimizer
from tensorflow.keras.optimizers import RMSprop
optimizer = RMSprop(learning_rate=0.001)  # RMSprop with a lower learning rate
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Step 3: Train the model with more epochs
history = model.fit(X_train, y_train, batch_size=16, epochs=1000, validation_data=(X_test, y_test))

# Step 4: Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Step 5: Evaluate the model
# Calculate MAE, MSE, RMSE, and MAPE
mae = mean_absolute_error(y_test_actual, predictions)
mse = mean_squared_error(y_test_actual, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Epoch 1/1000


c:\Users\dhevi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 351ms/step - loss: 0.0599 - val_loss: 0.3222
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0270 - val_loss: 0.1974
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0201 - val_loss: 0.0668
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0136 - val_loss: 0.0095
Epoch 5/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0109 - val_loss: 0.0042
Epoch 6/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0100 - val_loss: 0.0196
Epoch 7/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0071 - val_loss: 0.0269
Epoch 8/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0058 - val_loss: 0.0394
Epoch 9/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0059 - val_loss: 0.0317
Epoch 10/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0049 - val_loss: 0.0453
Epoch 11/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0047 - val_loss: 0.0312
Epoch 12/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0032 - val_lo

In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Step 1: Load and preprocess the data
data = pd.read_excel('IND.xlsx')
time_series = data[['year', 'debt']].set_index('year')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Increase sequence length to capture more historical context
seq_length = 10  # Experiment with different values
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 2: Build an improved LSTM model
model = Sequential()

# First LSTM layer with more units and return sequences
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2, seed=seed))  # Dropout to prevent overfitting

# Second LSTM layer
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2, seed=seed))  # Dropout to prevent overfitting

# Dense layer for output
model.add(Dense(1))

# Compile the model with RMSprop optimizer
from tensorflow.keras.optimizers import RMSprop
optimizer = RMSprop(learning_rate=0.001)  # RMSprop with a lower learning rate
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Step 3: Train the model with more epochs
history = model.fit(X_train, y_train, batch_size=16, epochs=500, validation_data=(X_test, y_test))


# Step 4: Make predictions for both training and testing data
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Inverse transform the predictions and actual values to original scale
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)

y_train_actual = scaler.inverse_transform(y_train)
y_test_actual = scaler.inverse_transform(y_test)

# Step 5: Evaluate the model on training data
# Calculate MAE, MSE, RMSE, and MAPE for training data
train_mae = mean_absolute_error(y_train_actual, train_predictions)
train_mse = mean_squared_error(y_train_actual, train_predictions)
train_rmse = np.sqrt(train_mse)
train_mape = np.mean(np.abs((y_train_actual - train_predictions) / y_train_actual)) * 100

# Step 6: Evaluate the model on testing data
# Calculate MAE, MSE, RMSE, and MAPE for testing data
test_mae = mean_absolute_error(y_test_actual, test_predictions)
test_mse = mean_squared_error(y_test_actual, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mape = np.mean(np.abs((y_test_actual - test_predictions) / y_test_actual)) * 100

# Print evaluation metrics for training data
print("Training Data Evaluation:")
print(f'Mean Absolute Error (MAE): {train_mae}')
print(f'Mean Squared Error (MSE): {train_mse}')
print(f'Root Mean Squared Error (RMSE): {train_rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {train_mape:.2f}%')

# Print evaluation metrics for testing data
print("\nTesting Data Evaluation:")
print(f'Mean Absolute Error (MAE): {test_mae}')
print(f'Mean Squared Error (MSE): {test_mse}')
print(f'Root Mean Squared Error (RMSE): {test_rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {test_mape:.2f}%')

Epoch 1/500


c:\Users\dhevi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 327ms/step - loss: 0.0711 - val_loss: 0.3800
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0335 - val_loss: 0.2611
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0247 - val_loss: 0.1884
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0214 - val_loss: 0.1389
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0183 - val_loss: 0.0811
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0149 - val_loss: 0.0439
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0132 - val_loss: 0.0173
Epoch 8/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0106 - val_loss: 0.0050
Epoch 9/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0076 - val_loss: 0.0107
Epoch 10/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0046 - val_loss: 0.0272
Epoch 11/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0043 - val_loss: 0.0402
Epoch 12/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0050 - val_loss: 0.0298
